In [48]:
import pandas as pd

file_name = "HSNCC_singleR_labels.csv"
df_singleR = pd.read_csv(file_name, index_col=0)
file_name = "HSNCC_scMRMA_labels.csv"
df_panglaodb = pd.read_csv(file_name, index_col=0)
# file_name = "ISCBgastric_scMRMA_TcellAI_labels_mnn.csv"
# df_TcellAI = pd.read_csv(file_name)




In [49]:
df_singleR['cluster'].value_counts()

Macrophages          698
Endothelial cells    690
Fibroblasts          652
Keratinocytes        503
DC                   501
Monocytes            426
Epithelial cells     391
B-cells              347
Erythrocytes         326
CD8+ T-cells         296
HSC                  269
Neutrophils          252
CD4+ T-cells         236
NK cells             215
Name: cluster, dtype: int64

In [11]:
df_panglaodb['cluster'].value_counts()

Plasmacytoid dendritic cells    78
Monocytes                       56
Mast cells                      54
Gamma delta T cells             51
B cells                         49
Macrophages                     43
Dendritic cells                 35
NK cells                        26
T cells                         26
Plasma cells                    22
Name: cluster, dtype: int64

In [50]:
# df_singleR[df_singleR['avg_log2FC'] > 2]

df_singleR = df_singleR.loc[(df_singleR['avg_log2FC'] >1.2) & (df_singleR['pct.1'] > 0.5)]
df_panglaodb = df_panglaodb.loc[(df_panglaodb['avg_log2FC'] >1.2) & (df_panglaodb['pct.1'] > 0.5)]
# df_TcellAI = df_TcellAI.loc[(df_TcellAI['avg_log2FC'] >1.2) & (df_TcellAI['pct.1'] > 0.5)]

In [51]:
unique_clusters1 = df_singleR["cluster"].drop_duplicates()

#process panglaodb
df_panglaodb.loc[df_panglaodb['cluster'] == 'B cells', 'cluster'] = 'B-cells'
df_panglaodb.loc[df_panglaodb['cluster'] == 'Fibroblast', 'cluster'] = 'Fibroblasts'
unique_clusters2 = df_panglaodb["cluster"].drop_duplicates()

# #process TcellAI
# df_TcellAI.loc[df_TcellAI['cluster'] == 'CD8_T', 'cluster'] = 'CD8+ T-cells'
# df_TcellAI.loc[df_TcellAI['cluster'] == 'CD4_naive', 'cluster'] = 'CD4+ T-cells'
# unique_clusters3 = df_TcellAI["cluster"].drop_duplicates()

In [28]:
merged_clusters = pd.concat([unique_clusters1, unique_clusters2])
merged_clusters = merged_clusters.drop_duplicates()

merged_gene = pd.concat([df_singleR["gene"], df_panglaodb["gene"]])
merged_gene = merged_gene.drop_duplicates()

subcellType_data = {
    'SubCellTypeId': range(1, len(merged_clusters) + 1),
    'SubCellTypeName': merged_clusters,
    'CellTypeId': [0] * len(merged_clusters) 
}


gene_data = {
    'GeneId': range(1, len(merged_gene) + 1),
    'GeneName': merged_gene
}




df_subcellType = pd.DataFrame(subcellType_data)
df_subcellType.to_csv('subcellType.csv', index=False)


df_gene = pd.DataFrame(gene_data)
df_gene.to_csv('gene.csv', index=False)

In [31]:
df_singleR

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
S100A9,0.000000,5.477661,0.955,0.316,0.0,Monocytes,S100A9
S100A8,0.000000,5.260419,0.930,0.291,0.0,Monocytes,S100A8
LYZ,0.000000,3.899506,0.975,0.281,0.0,Monocytes,LYZ
S100A12,0.000000,3.885766,0.677,0.034,0.0,Monocytes,S100A12
RP11-1143G9.4,0.000000,3.712485,0.607,0.073,0.0,Monocytes,RP11-1143G9.4
...,...,...,...,...,...,...,...
VMP1.1,0.005825,1.308173,1.000,0.417,1.0,Fibroblasts,VMP1
EIF4EBP1.2,0.006208,1.315462,0.750,0.211,1.0,Fibroblasts,EIF4EBP1
SQSTM1.4,0.007087,1.682713,1.000,0.385,1.0,Fibroblasts,SQSTM1
ANXA2.13,0.008922,1.336477,1.000,0.594,1.0,Fibroblasts,ANXA2


In [30]:
df_combine = pd.concat([df_singleR[["gene", 'cluster']], df_panglaodb[["gene", 'cluster']]])
df_combine = df_combine.drop_duplicates()
mapped_df = df_combine.merge(df_subcellType, left_on='cluster', right_on='SubCellTypeName', how='left')
subcellType_gene_df = mapped_df.merge(df_gene, left_on='gene', right_on='GeneName', how='left')
subcellType_gene_df = subcellType_gene_df[['SubCellTypeId', 'GeneId']]
subcellType_gene_df.to_csv('subcellType_gene.csv', index=False)


In [35]:
df_singleR.groupby('gene')[['p_val', 'avg_log2FC']].max()

,p_val,avg_log2FC
gene,,
A1BG,2.173672e-03,1.337272
A2M,1.767849e-14,2.710849
AC092580.4,0.000000e+00,1.760375
ACKR3,2.146681e-222,1.514655
ACP1,2.166553e-03,1.443659
...,...,...
YWHAE,7.976460e-04,1.242622
YWHAH,5.143206e-05,1.719809
YWHAQ,5.926827e-06,1.205676


In [44]:
 df_singleR.groupby('gene')[[ 'p_val', 'avg_log2FC']].max()['']

,p_val,avg_log2FC
gene,,
A1BG,2.173672e-03,1.337272
A2M,1.767849e-14,2.710849
AC092580.4,0.000000e+00,1.760375
ACKR3,2.146681e-222,1.514655
ACP1,2.166553e-03,1.443659
...,...,...
YWHAE,7.976460e-04,1.242622
YWHAH,5.143206e-05,1.719809
YWHAQ,5.926827e-06,1.205676


In [52]:
df_singleR = df_singleR.groupby('gene')[[ 'p_val', 'avg_log2FC']].max()




df_singleR = df_singleR.merge(df_gene, left_on='gene', right_on='GeneName', how='left')
df_singleR['AnnId'] = '1'


df_panglaodb = df_panglaodb.groupby('gene')[['p_val', 'avg_log2FC']].max()
df_panglaodb = df_panglaodb.merge(df_gene, left_on='gene', right_on='GeneName', how='left')
df_panglaodb['AnnId'] = '2'



df_annotationTool_gene_data = pd.concat([df_singleR[['AnnId', 'GeneId', 'p_val', 'avg_log2FC']], 
                                         df_panglaodb[['AnnId', 'GeneId', 'p_val', 'avg_log2FC']]])

df_annotationTool_gene_data.to_csv('annotationTool_gene.csv', index=False)

In [55]:
df_singleR.loc[df_singleR['GeneName'] == 'CD2']

,p_val,avg_log2FC,GeneId,GeneName,AnnId
104,0.0,1.873349,124,CD2,1


In [54]:
df_singleR

,p_val,avg_log2FC,GeneId,GeneName,AnnId
0,2.173672e-03,1.337272,874,A1BG,1
1,1.767849e-14,2.710849,733,A2M,1
2,0.000000e+00,1.760375,126,AC092580.4,1
3,2.146681e-222,1.514655,681,ACKR3,1
4,2.166553e-03,1.443659,380,ACP1,1
...,...,...,...,...,...
874,7.976460e-04,1.242622,784,YWHAE,1
875,5.143206e-05,1.719809,768,YWHAH,1
876,5.926827e-06,1.205676,599,YWHAQ,1
877,2.544708e-05,1.297934,856,ZFAND3,1


In [23]:
df_gene['LitId'] = '1'
df_gene[['LitId', 'GeneId']].to_csv('literatrue_gene.csv', index=False)